In [89]:
import xlsxwriter
import json
import glob
import io
import os
from typing import Tuple
from PIL import Image

In [90]:
file = open('3087-large-medieval-house.json')
schema_json = json.loads(file.read())
file.close()

In [91]:
file = open('blocks_raw.json')
blocks_name = json.loads(file.read())
file.close()

In [92]:
blocks = {}
for item in blocks_name:
    blocks[item['index']] = item

In [93]:
workbook = xlsxwriter.Workbook(schema_json['title']+'.xlsx')

In [94]:
cells = schema_json['cells']

In [95]:
xs = []
ys = []
zs = []

In [96]:
for i in range(len(cells)):
    cell = cells[i]
    if cell['x'] not in xs:
        xs.append(cell['x'])
    if cell['y'] not in ys:
        ys.append(cell['y'])
    if cell['z'] not in zs:
        zs.append(cell['z'])

In [97]:
x_max = max(xs)
x_min = min(xs)
y_max = max(ys)
y_min = min(ys)
z_max = max(zs)
z_min = min(zs)
print(f'x_max: {x_max}, y_max: {y_max}, z_max: {z_max}, x_min: {x_min}, y_min: {y_min}, z_min: {z_min}')

x_max: 17, y_max: 20, z_max: 10, x_min: -18, y_min: 0, z_min: -10


In [98]:
ys.sort()
xs.sort()
zs.sort()
ys

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [99]:
y_sheets = {}

In [100]:
for i in range(len(ys)):
    worksheet = workbook.add_worksheet(str(ys[i]+1))
    y_sheets[ys[i]] = [worksheet,[]]
y_sheets    

{0: [<xlsxwriter.worksheet.Worksheet at 0x233966278e0>, []],
 1: [<xlsxwriter.worksheet.Worksheet at 0x23396626b90>, []],
 2: [<xlsxwriter.worksheet.Worksheet at 0x23396627ac0>, []],
 3: [<xlsxwriter.worksheet.Worksheet at 0x23396627f40>, []],
 4: [<xlsxwriter.worksheet.Worksheet at 0x233966276d0>, []],
 5: [<xlsxwriter.worksheet.Worksheet at 0x23396626d40>, []],
 6: [<xlsxwriter.worksheet.Worksheet at 0x23396626ec0>, []],
 7: [<xlsxwriter.worksheet.Worksheet at 0x23396627b80>, []],
 8: [<xlsxwriter.worksheet.Worksheet at 0x23396627be0>, []],
 9: [<xlsxwriter.worksheet.Worksheet at 0x23396627250>, []],
 10: [<xlsxwriter.worksheet.Worksheet at 0x23396627820>, []],
 11: [<xlsxwriter.worksheet.Worksheet at 0x23396627130>, []],
 12: [<xlsxwriter.worksheet.Worksheet at 0x23396627f70>, []],
 13: [<xlsxwriter.worksheet.Worksheet at 0x23391c46bc0>, []],
 14: [<xlsxwriter.worksheet.Worksheet at 0x233966269e0>, []],
 15: [<xlsxwriter.worksheet.Worksheet at 0x23396627ee0>, []],
 16: [<xlsxwriter.

In [101]:
left_border = workbook.add_format()
right_border = workbook.add_format()
top_border = workbook.add_format()
bottom_border = workbook.add_format()
bottom_border.set_bottom(5)
top_border.set_top(5)
left_border.set_left(5)
right_border.set_right(5)

In [102]:
def add_numbers_to_sheet(sheet):
    sheet[0].set_column_pixels(0, len(xs)+1, 20)
    for i in range(len(zs)+2):
        sheet[0].set_row_pixels(i,21)
    xs_range = list(range(1,len(xs)+1))
    sheet[0].write_row(0,1,xs_range,bottom_border)
    xs_range.reverse()
    sheet[0].write_row(len(zs)+1,1,xs_range,top_border)
    zs_range = list(range(1,len(zs)+1))
    sheet[0].write_column(1,0,zs_range,right_border)
    zs_range.reverse()
    sheet[0].write_column(1,len(xs)+1,zs_range,left_border)
        

In [103]:
for y in ys:
    add_numbers_to_sheet(y_sheets[y])

In [104]:
def buffer_image(image: Image, format: str = 'PNG'):
    # Store image in buffer, so we don't have to write it to disk.
    buffer = io.BytesIO()
    image.save(buffer, format=format)
    return buffer, image

In [105]:
def open_img(path: str):
    image = Image.open(path)
    return image

In [106]:
def rotate(image: Image, rotation: int = 90, format='PNG'):
    image = image.rotate(rotation, Image.NEAREST, expand=1)
    return buffer_image(image, format)

In [107]:
resources = {}

In [108]:
def add_block(x,y,z,id,data=0):
    x = abs(x_min)+x+1
    z = abs(z_min)+z+1
    sheet = y_sheets[y][0]
    sheet.write(z,x,id)
    sheet.write_comment(z,x,blocks[id]['name'])
    name = blocks[id]['name']
    if name.startswith("double_"):
        name = name[7:]
    if name not in resources.keys():
        resources[name] = 0
    if(blocks[id]['type']=="Door" and data==2):
        resources[name]-=1
    resources[name] += 1
    if(blocks[id]['name'].startswith("double_")):
        resources[name] += 1    
    if 'sprite' in blocks[id].keys():
        sheet.insert_image(z,x, blocks[id]['sprite'], {'x_scale': 1, 'y_scale': 1})
    if blocks[id]['type'] == "Slab" and (data==5 or data==8 or data == 0):
        sheet.insert_image(z,x, blocks[id]['downsprite'], {'x_scale': 1, 'y_scale': 1})
    if blocks[id]['type'] == "Slab" and (data==11 or data==12):
        sheet.insert_image(z,x, blocks[id]['upsprite'], {'x_scale': 1, 'y_scale': 1})
    if blocks[id]['type'] == "Wood" and data==4:
        image_buffer, image = rotate(open_img(blocks[id]['upsprite']), 90, format='PNG')
        sheet.insert_image(z,x, blocks[id]['upsprite'], {'image_data': image_buffer, **{'x_scale': 1.3, 'y_scale': 1.3}})
    elif blocks[id]['type'] == "Wood" and (data==0 or data==5 or data==8):
        sheet.insert_image(z,x, blocks[id]['downsprite'], {'x_scale': 1, 'y_scale': 1})   

In [109]:
for cell in cells:
    if 'data' in cell.keys():
        add_block(cell['x'],cell['y'],cell['z'],cell['block_id'],cell['data'])
    else:
        add_block(cell['x'],cell['y'],cell['z'],cell['block_id'],cell['data']) 
    if(cell['block_id'] not in y_sheets[cell['y']][1]):
        y_sheets[cell['y']][1].append(cell['block_id'])

C:\Users\dbury\AppData\Local\Temp\ipykernel_16692\1718297770.py:2: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  image = image.rotate(rotation, Image.NEAREST, expand=1)


In [110]:
def number_to_letter(numb):
    file = open("letters.json")
    letters = json.load(file)
    file.close()
    result = ''
    n = numb
    while n != 0:
        n, d = divmod(n, 26)
        result+= letters[str(d)]
    return result[::-1]

In [111]:
formula_var = '=COUNTIF(B2:'+number_to_letter(len(xs)+1)+str(len(zs)+1)+','+number_to_letter(len(xs)+6)
formula_empty = '=COUNTBLANK(B2:'+number_to_letter(len(xs)+1)+str(len(zs)+1)+')'
formula_sum = '=SUM('+number_to_letter(len(xs)+7)+'3:'+number_to_letter(len(xs)+7)

In [112]:
type_offset = 4
id_offset = 5
count_offset = 6
image_offset = 3

In [113]:
for y in ys:
    [sheet, ids] = y_sheets[y]
    sheet.set_column(len(xs)+id_offset,len(xs)+count_offset, 4)
    sheet.set_column_pixels(len(xs)+image_offset,len(xs)+image_offset, 20)
    sheet.write(1,len(xs)+type_offset,'type')
    sheet.write(1,len(xs)+id_offset,'id')
    sheet.write(1,len(xs)+count_offset,'count')
    in_counter = 0
    max_type_lenght = 0
    for id in ids:
        in_counter += 1
        if 'sprite' in blocks[id].keys():
            sheet.insert_image(1+in_counter,len(xs)+image_offset, blocks[id]['sprite'], {'x_scale': 1, 'y_scale': 1})
        if (blocks[id]['type'] == 'Slab' or blocks[id]['type'] == 'Wood') and 'downsprite' in blocks[id].keys() :
            sheet.insert_image(1+in_counter,len(xs)+image_offset, blocks[id]['downsprite'], {'x_scale': 1, 'y_scale': 1})
        max_type_lenght = max(max_type_lenght,len(blocks[id]['name']))
        sheet.write(1+in_counter,len(xs)+type_offset,blocks[id]['name'])
        sheet.write(1+in_counter,len(xs)+id_offset,id)
        sheet.write(1+in_counter,len(xs)+count_offset,formula_var+str(in_counter+2)+')')
    sheet.set_column(len(xs)+type_offset,len(xs)+type_offset, max_type_lenght)
    in_counter += 1
    sheet.write(1+in_counter,len(xs)+type_offset,'air')
    sheet.write(1+in_counter,len(xs)+count_offset,formula_empty)
    in_counter += 1
    sheet.write(1+in_counter,len(xs)+type_offset,'sum')
    sheet.write(1+in_counter,len(xs)+count_offset,formula_sum+str(in_counter+1)+')')
    in_counter += 1
    sheet.write(1+in_counter,len(xs)+type_offset,'square')
    sheet.write(1+in_counter,len(xs)+count_offset,len(xs)*len(zs))


    

In [114]:
resources = dict(sorted(resources.items(), key=lambda x:x[1],reverse=True))
resources

{'Oak_Wood_Planks': 532,
 'Stone_Brick': 510,
 'Wood': 326,
 'acacia_stairs': 305,
 'wooden_slab': 297,
 'Dirt': 279,
 'Cobblestone': 258,
 'Grass': 244,
 'Stone_Slab': 108,
 'Cobblestone_Wall': 108,
 'stone_brick_stairs': 77,
 'Fence': 56,
 'stained_glass_pane': 55,
 'Double_Stone_Slab': 36,
 'Oak_Wood_Stairs': 32,
 'Trapdoor': 17,
 'Rails': 10,
 'Glowstone': 4,
 'jungle_stairs': 2,
 'Wooden_Door': 2,
 'Stone_Button': 1}

In [115]:
summary = workbook.add_worksheet("summary")
summary.write(0,1,"type")
summary.write(0,2,"amount")
summary.set_column_pixels(0, 0, 20)

0

In [116]:
def get_block_from_dict(name):
    for id, block in blocks.items():
        if(block['name'] == name):
            return block
    print(name)        

In [117]:
counter = 0
for (id, amount) in resources.items():
    counter+=1
    summary.set_row_pixels(counter, 21)
    summary.write(counter, 1, id)
    summary.write(counter, 2, amount)
    block = get_block_from_dict(id)
    if 'sprite' in block.keys():
        summary.insert_image(counter,0, block['sprite'], {'x_scale': 1, 'y_scale': 1})
    if (block['type'] == 'Slab' or block['type'] == 'Wood') and 'downsprite' in block.keys() :
        summary.insert_image(counter,0, block['downsprite'], {'x_scale': 1, 'y_scale': 1})
        

In [118]:
workbook.close()